##install

In [6]:
pip install keybert


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [9]:
pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.5 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.21.0-py3-none-any.whl size=35593192 sha256=8a48dbae8124b69920b6f5e8ac95fa7041d185a9de065812b105c43e549b82f4
  Stored in directory: /root/.cache/pip/wheels/b0/16/3d/95053ab5298f0f0f22ffea6de0200b6f24bffb73cab4c1a828
Successfully built kiwipiepy_model


In [10]:
pip install transformers

##키워드 추출

In [12]:
#@title 키워드 및 중요도 확인
#BERT 임베딩(여기선 KoBERT)을 활용해, 명사들 중에서 가장 문맥상 중요한 단어를 추출하는 역할 = kiwi로 추출한 명사들에 중요도 부여
from keybert import KeyBERT
# kiwi = 한국어 문장에서 "명사"만 뽑는 형태소 분석기
from kiwipiepy import Kiwi
#KeyBERT에 한국어 특화 임베딩을 제공
from transformers import BertModel


filename = "input.txt"



with open(filename, encoding="utf-8") as f:
    text = f.read()

# 2. KoBERT 모델 & KeyBERT 객체
model = BertModel.from_pretrained('skt/kobert-base-v1') #표준 한국어 임베딩 백본 사용
kw_model = KeyBERT(model)

# 3. Kiwi로 명사만 추출
kiwi = Kiwi()
nouns_list = []
for sentence in kiwi.analyze(text):
    nouns = [token.form for token in sentence[0] if token.tag.startswith('NN')]
    if nouns:
        nouns_list.extend(nouns)
result_text = ' '.join(nouns_list)

# 4. KeyBERT로 키워드 추출
keywords = kw_model.extract_keywords(
    result_text,
    keyphrase_ngram_range=(1, 1),   # 1단어만 추출. (2,3 등도 가능)
    stop_words=None,
    top_n=20
)
print(keywords)


[('체이서', 0.6967), ('팔라디스', 0.6962), ('이아나', 0.695), ('실랑이', 0.6805), ('번째', 0.6776), ('수도', 0.6587), ('상태', 0.6546), ('애물단지', 0.6533), ('그사이', 0.6492), ('활약', 0.6428), ('남자', 0.6367), ('도뮬릿가', 0.6363), ('대로', 0.6306), ('응시', 0.6303), ('평생', 0.6296), ('아가씨', 0.6234), ('부스러기', 0.6187), ('지하', 0.6127), ('제압', 0.6049), ('출소', 0.6035)]


In [13]:
#@title 키워드 중요순서대로 txt 파일 저장
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

filename = "input.txt"

with open(filename, encoding="utf-8") as f:
    text = f.read()

# 2. KoBERT 모델 & KeyBERT 객체
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

# 3. Kiwi로 명사만 추출
kiwi = Kiwi()
nouns_list = []
for sentence in kiwi.analyze(text):
    nouns = [token.form for token in sentence[0] if token.tag.startswith('NN')]
    if nouns:
        nouns_list.extend(nouns)
result_text = ' '.join(nouns_list)

# 4. KeyBERT로 키워드 추출
keywords = kw_model.extract_keywords(
    result_text,
    keyphrase_ngram_range=(1, 1),
    stop_words=None,
    top_n=20
)

# 5. 키워드만 추출(중요도 순서대로), 쉼표로 연결
keywords_only = [word for word, score in keywords]
keywords_line = ','.join(keywords_only)

# 6. 파일로 저장
with open("output_keywords.txt", "w", encoding="utf-8") as f:
    f.write(keywords_line)

print("키워드 추출 완료! => output_keywords.txt")
print(keywords_line)


키워드 추출 완료! => output_keywords.txt
체이서,팔라디스,이아나,실랑이,번째,수도,상태,애물단지,그사이,활약,남자,도뮬릿가,대로,응시,평생,아가씨,부스러기,지하,제압,출소
